In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pylab as plt

import sys, os
sys.path.append(os.pardir)

# 4章 ニューラルネットワークの学習

ここで言う「学習」とは、訓練データから最適な重みパラメータの値を自動で獲得すること

<img  src="images/K34SRVI5L1W6BKEPF9XGQEFLG4XM4VBL.png"/>

## 4.1 データから学習する

### 4.1.1 データ駆動

* ニューラルネットワークの利点は、すべての問題を同じ流れで解くことができるということ
    * 手書き文字や人やモノによって異なる特徴量を自動的に抽出する

### 4.1.2 訓練データとテストデータ

* 訓練データ（教師データ）を使って最適なパラメータを探索
* テストデータを使って訓練したモデルの実力を評価
* 特定のデータセットに過度に対応した状態を過学習(overfitting)という

## 4.2 損失関数

* 損失関数(loss function): ニューラルネットワークの性能の"悪さ"を示す指標
    * この値が最小になるようなパラメータを探索する

### 4.2.1 2乗和誤差

$$
    E = \frac{1}{2} \sum_k (y_k - t_k)^2
$$

* $y_k$: ニューラルネットワークの出力
* $t_k$: 教師データ
* $k$: データの次元数

In [2]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [3]:
# 「2」を正解とする（one-hot表現）
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [4]:
# 例1：「2」の確率が最も高い場合（0.6）
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

mean_squared_error(np.array(y), np.array(t))

0.097500000000000031

In [5]:
# 例2：「7」の確率が最も高い場合（0.6）
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

mean_squared_error(np.array(y), np.array(t))

0.59750000000000003

### 4.2.2 交差エントロピー誤差

$$
    E = - \sum_k t_k \log y_k
$$

In [6]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

`np.log(0)`がマイナス無限大となって計算を続けられなくなるため、微小な値`delta`を足している。

In [7]:
# 「2」を正解とする（one-hot表現）
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [8]:
# 例1：「2」の確率が最も高い場合（0.6）
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

cross_entropy_error(np.array(y), np.array(t))

0.51082545709933802

In [9]:
# 例2：「7」の確率が最も高い場合（0.6）
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

cross_entropy_error(np.array(y), np.array(t))

2.3025840929945458

### 4.2.3 ミニバッチ学習

$$
    E = - \frac{1}{N} \sum_n \sum_k t_{nk} \log y_{nk}
$$

* $N$: 訓練データの個数

$N$個のデータを使って平均の損失関数を求めることで、データ個数によらない統一した指標になる。また、大量のデータすべてを使わなくても、一部のデータ（ミニバッチ）を使って、全体の近似値とすることができる。

In [10]:
from dataset.mnist import load_mnist

In [11]:
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000, 10)

(60000, 784)
(60000, 10)


60000個のデータから10個をランダムに選ぶ。

In [12]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [13]:
print(x_batch.shape)
print(t_batch.shape)

(10, 784)
(10, 10)


In [14]:
np.random.choice(60000, 10)

array([41725, 57009,  4099, 43940, 10203, 53474, 56345, 27819, 47041, 35139])

### 4.2.4 ［バッチ対応版］交差エントロピー誤差の実装

データ数が1個でもN個でも対応できるようにした交差エントロピー誤差の実装

In [15]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

教師データ`t`がone-hot表現ではなくラベル（「2」や「7」といった値）として与えられた場合

In [16]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

In [17]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

```python
t = [2, 7, 0, 9, 4]
y[np.arange(batch_size), t]
# => [y[0,2], y[1,7], y[2,0], y[3,9], y[4,4]]
```

one-hot表現で`t`の値が0となる要素は、交差エントロピー誤差も0となるので無視してよい。

### 4.2.5 なぜ損失関数を設定するのか？

認識精度が高くなるようにパラメータを調節するのではなく、損失関数が0になるようにパラメータを調節するのはなぜか。

パラメータの微小な変化で、認識精度はほとんど変わらず変わったとしても不連続なとびとびの値となる。対して、損失関数は連続的な変化を見せるので、微分によってパラメータの収束する方向を決められる。

## 4.3 数値微分

### 4.3.1 微分

中心差分による実装

$$
    \frac{d f(x)}{d x} = \lim_{h \rightarrow 0} \frac{f(x+h) - f(x-h)}{2h}
$$

In [18]:
def numerical_diff(f, x):
    h = 1e-4 # 0.0001
    return (f(x+h) - f(x-h)) / (2*h)

### 4.3.2 数値微分の例

$$
    f(x) = 0.01 x^2 + 0.1 x
$$

In [19]:
def function_1(x):
    return 0.01*x**2 + 0.1*x

In [20]:
numerical_diff(function_1, 5)

0.1999999999990898

In [21]:
numerical_diff(function_1, 10)

0.2999999999986347

$x$で微分した式は以下の通り。

$$
    \frac{d f(x)}{d x} = 0.02 x + 0.1
$$

真の微分は$x=5$のとき$0.2$、$x=10$のとき$0.3$となるので、数値微分の値と一致する。

## 4.4 勾配

$$
    f(x_0, x_1) = x_0^2 + x_1^2
$$

In [23]:
def function_2(x):
    return x[0]**2 + x[1]**2

$x$の各要素について偏微分して、関数の値が小さくなる方向を求める。これを勾配と呼ぶ。

In [22]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x と同じ形状の配列を生成

    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h) の計算
        x[idx] = tmp_val + h
        fxh1 = f(x)

        # f(x-h) の計算
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 値を元に戻す

    return grad

In [24]:
numerical_gradient(function_2, np.array([3.0, 4.0]))

array([ 6.,  8.])

In [25]:
numerical_gradient(function_2, np.array([0.0, 2.0]))

array([ 0.,  4.])

In [26]:
numerical_gradient(function_2, np.array([3.0, 0.0]))

array([ 6.,  0.])

### 4.4.1 勾配法

$$
\begin{eqnarray*}
    x_0 &=& x_0 - \eta \frac{\partial f}{\partial x_0} \\
    x_1 &=& x_1 - \eta \frac{\partial f}{\partial x_1}
\end{eqnarray*}
$$

学習率$\eta$の勾配だけ$x$を更新する。

In [27]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x

    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad

    return x

In [28]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

array([ -6.11110793e-10,   8.14814391e-10])

In [29]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100)

array([-2.99999994,  3.99999992])

学習率が大きすぎると大きな値へ発散してしまう。学習率が小さすぎるとなかなか収束しない。

学習率のようなあらかじめ決めておくパラメータをハイパーパラメータと呼ぶ。

### 4.4.2 ニューラルネットワークに対する勾配

$$
\begin{eqnarray*}
    W &=& \left( \begin{array}{ccc}
        w_{11} & w_{21} & w_{31} \\
        w_{12} & w_{22} & w_{32}
    \end{array} \right) \\
    \frac{\partial L}{\partial W} &=& \left( \begin{array}{ccc}
        \frac{\partial L}{\partial w_{11}} & \frac{\partial L}{\partial w_{21}} & \frac{\partial L}{\partial w_{31}} \\
        \frac{\partial L}{\partial w_{12}} & \frac{\partial L}{\partial w_{22}} & \frac{\partial L}{\partial w_{32}}
    \end{array} \right)
\end{eqnarray*}
$$

* $W$: 重み
* $L$: 損失関数

In [31]:
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

In [32]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # ガウス分布で初期化

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

In [33]:
net = simpleNet()
print(net.W) # 重みパラメータ

[[-0.69441465  0.95391874 -1.18892333]
 [-0.94960467 -0.2353381   0.92157066]]


In [34]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[-1.27129299  0.36054696  0.1160596 ]


In [35]:
np.argmax(p) # 最大値のインデックス

1

In [36]:
t = np.array([0, 0, 1]) # 正解ラベル
net.loss(x, t)

0.92691493529313818

In [37]:
def f(W):
    # 引数Wはダミー
    return net.loss(x, t)

In [38]:
dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.05930313  0.30323318 -0.36253631]
 [ 0.0889547   0.45484977 -0.54380447]]


## 4.5 学習アルゴリズムの実装

1. ミニバッチ
1. 勾配の算出
1. パラメータの更新
1. 1-3を繰り返す

確率的勾配降下法(SGD: stochastic gradient descent)

### 4.5.1 2層ニューラルネットワークのクラス

In [39]:
from common.functions import *
from common.gradient import numerical_gradient

In [44]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
                 weight_init_std=0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

In [45]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape) # (784, 100)
print(net.params['b1'].shape) # (100,)
print(net.params['W2'].shape) # (100, 10)
print(net.params['b2'].shape) # (10,)

(784, 100)
(100,)
(100, 10)
(10,)


In [46]:
x = np.random.rand(100, 784) # ダミーの入力データ（100 枚分）
y = net.predict(x)

In [50]:
y[0]

array([ 0.10010116,  0.10318017,  0.10217018,  0.09856556,  0.09637222,
        0.10098556,  0.098416  ,  0.10155795,  0.10827452,  0.0903767 ])